#Reverse Mode Automatic Differentiation (AD)

Dynamic Reverse mode AD can be implemented by declaring a class `Var` to represent a value and the child expressions that the value depends on. We've provided the implementation that was shown in the lecture slides. 

Tasks:

1. Complete Addition (`__add__`) method below. 
2. Complete division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [ ]:
import math

class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None #Initialize to None, which means it's not yet evaluated

    def grad(self):
        #recurse only if the value is not yet cached
        if self.grad_value is None:
            #calculate derivative using chain rule
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value
    
    def __str__(self):
        return str(self.value)

    def __mul__(self, other):

      z = Var(self.value * other.value)
    
      #weight = dz/dself=other.value
      self.children.append((other.value, z))
    
      #weight = dz/dother=self.value
      other.children.append((self.value, z))
      return z

# For a=x*y a is a new Var that is a child of both x and y

    def __add__(self, other):
      #your code here
      z = Var(self.value + other.value)
      
      #weight = dz/dself=1
      self.children.append((1, z))
    
      #weight = dz/dother=1
      other.children.append((1, z))
      return z

    #Add other functions as well

    def __sub__(self, other):
      #your code here
      z = Var(self.value - other.value)
      
      #weight = dz/dself=other.value
      self.children.append((1, z))
    
      #weight = dz/dother=self.value
      other.children.append((-1, z))
      return z

    def __truediv__(self, other):
      #your code here
      z = Var(self.value / other.value)
      
      #weight = dz/dself=other.value
      self.children.append((1/other.value, z))

      #weight = dz/dother=self.value
      other.children.append(((-self.value/other.value**2), z))
      return z


def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z

In [ ]:
# Tests

print(Var(1) + Var(1))
print(Var(1) + Var(1) / Var(1) - Var(1))
# print(Var(1) + Var(1) / Var(1) - Var(1)**Var(1))


2
1.0


#Forward computation first
1. Try running the following code to compute the value of the function  $z=a+b, a=x*y, b =sin(x)$  given  $x=0.5$  and  $y=4.2$.
2. Print out the children of x,y,a,b with their derivatives $\frac{\partial a}{\partial x},\frac{\partial b}{\partial x},\frac{\partial a}{\partial y},\frac{\partial z}{\partial a},\frac{\partial z}{\partial b}$


In [ ]:
x=Var(0.5)
y=Var(4.2)

a=x*y
b=sin(x)
z=a+b
z.grad_value=1 

#Complete the code here
print(f'Children of x is {x.children}...and its derivative is ')
print(f'Children of y is {y.children}...and its derivative is ')

Children of x is [(4.2, <__main__.Var object at 0x7facae6af290>), (0.8775825618903728, <__main__.Var object at 0x7facae6af550>)]...and its derivative is 
Children of y is [(0.5, <__main__.Var object at 0x7facae6af290>)]...and its derivative is 


#Reverse mode computation

So far we have done forward computing as we go. But we haven't computed $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$ which is what we want essentially.

1. Run the code below
2. Print out the gradient of each variable and complete the code

In [ ]:
z.grad_value=1  
z.grad_value = 1.0 #Note that we have to 'seed' the gradient of z to 1 (e.g. ∂z/∂z=1) before computing grads

print('z:', z)
print("dz/dx: ",x.grad())

#Complete the code here
print("dz/dy: ",y.grad())



#Test your functions
assert abs(z.value - 2.579425538604203) <= 1e-15
assert abs(x.grad() - (y.value + math.cos(x.value))) <= 1e-15
assert abs(y.grad() - x.value) <= 1e-15

z: 2.579425538604203
dz/dx:  5.077582561890373
dz/dy:  0.5
